In [1]:
import numpy as np 
import tensorflow as tf 
import matplotlib.pyplot as plt
import random
from tensorflow.examples.tutorials.mnist import input_data

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
%matplotlib inline

data preprocess

In [3]:
mnist = input_data.read_data_sets("MNIST_data/" , one_hot = True)

train_data = mnist.train
valid_data = mnist.validation
test_data = mnist.test

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
train_image = np.reshape(train_data.images , [-1,28,28,1])
valid_image = np.reshape(valid_data.images , [-1,28,28,1])
test_image = np.reshape(test_data.images , [-1,28,28,1])
train_image = train_image / 255
valid_image = valid_image / 255
test_image = test_image / 255
train_label = train_data.labels
valid_label = valid_data.labels
test_label = test_data.labels

build graph

In [5]:
def add_dense(input_layer , size , activation=None):
    Weights = tf.Variable(tf.truncated_normal(shape=size , stddev=0.1))
    Biases = tf.Variable(tf.constant(0.1 , shape = [size[1]]))
    if activation is None:
        return tf.matmul(input_layer , Weights) + Biases
    else:
        return activation( tf.matmul(input_layer , Weights) + Biases )

In [6]:
def add_conv2D(input_layer , shape):
    Weights = tf.Variable(tf.truncated_normal(shape=shape , stddev=0.1))
    Biases = tf.Variable(tf.constant(0.1 , shape = [shape[3]])) 
    conv2D = tf.nn.conv2d(input=input_layer , filter=Weights , strides=[1,1,1,1] , padding="SAME")
    return tf.nn.relu(conv2D + Biases)

In [7]:
def add_pool(input_layer , size):
    return tf.nn.max_pool(input_layer , ksize=[1,size,size,1] , strides=[1,2,2,1] , padding="SAME")

In [8]:
def evaluate(y , y_):
    correct_prediction = tf.equal(tf.argmax(y, 1),tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    return accuracy

In [9]:
#input
X = tf.placeholder(tf.float32 , shape=[None , 28 ,28 ,1])
y = tf.placeholder(tf.int32 , shape=[None , 10])
keep_prob = tf.placeholder(tf.float32 , shape= None )  
#conv layer
conv_1 = add_conv2D(X , shape=[5,5,1,32]) #patch 5x5 , in size = 1 , out size = 32 -------- output size = 28x28x1x32
pool_1 = add_pool(conv_1 , size = 2) #output_size = 14x14x1x32
conv_2 = add_conv2D(pool_1 , shape=[5,5,32,64]) #patch 5x5 , in size = 32 , out size = 64 -------- output size = 14x14x1x64
pool_2 = add_pool(conv_2 , size = 2) #output_size = 7x7x1x64

#flatten
pool_2_flat = tf.reshape(pool_2 , [-1,7*7*64])

#fully connected layer
fc_1 = add_dense(pool_2_flat , [7*7*64 , 1024] , tf.nn.relu)
fc_1_drop = tf.nn.dropout(fc_1, keep_prob)
fc_2 = add_dense(fc_1_drop , [1024 , 10])
predict = tf.nn.softmax(fc_2)
accuracy = evaluate(y , predict)

In [10]:
loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=y , logits=fc_2)
        )

optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

training

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

epochs = 100
batch_size =256


for i in range(epochs):
    print("Epoch %2d/%2d: "%(i+1,epochs))
    
    index = [i for i in range(train_image.shape[0])]
    random.shuffle(index)
    
    while len(index)>0:
        index_size = len(index)
        batch_index=[index.pop() for _ in range(min(batch_size,index_size))]    
        feed_dict = {
            X: train_image[batch_index,:], 
            y: train_label[batch_index], 
            keep_prob : 0.5
        }
        sess.run(optimizer,feed_dict=feed_dict)
    
    train_loss = sess.run(loss,feed_dict={
        X : train_image,
        y : train_label,
        keep_prob : 1
    })
    valid_loss = sess.run(loss,feed_dict={
        X : valid_image,
        y : valid_label,
        keep_prob : 1
    })
    train_acc = sess.run(accuracy,feed_dict={
        X : train_image,
        y : train_label,
        keep_prob : 1
    })
    valid_acc = sess.run(accuracy,feed_dict={
        X : valid_image,
        y : valid_label,
        keep_prob : 1
    })
    print("train loss = %.4f , train accuracy = %.4f , valid loss = %.4f , valid accuracy = %.4f"%(train_loss,train_acc,valid_loss,valid_acc))


Epoch  1/100: 
train loss = 2.3068 , train accuracy = 0.1123 , valid loss = 2.3072 , valid accuracy = 0.1126
Epoch  2/100: 
train loss = 2.2957 , train accuracy = 0.1123 , valid loss = 2.2959 , valid accuracy = 0.1126
Epoch  3/100: 
train loss = 0.6713 , train accuracy = 0.8082 , valid loss = 0.6501 , valid accuracy = 0.8128
Epoch  4/100: 
train loss = 0.3022 , train accuracy = 0.9082 , valid loss = 0.2828 , valid accuracy = 0.9172
Epoch  5/100: 
train loss = 0.2202 , train accuracy = 0.9346 , valid loss = 0.2042 , valid accuracy = 0.9394
Epoch  6/100: 
train loss = 0.1728 , train accuracy = 0.9480 , valid loss = 0.1620 , valid accuracy = 0.9512
Epoch  7/100: 
train loss = 0.1501 , train accuracy = 0.9540 , valid loss = 0.1417 , valid accuracy = 0.9576
Epoch  8/100: 
train loss = 0.1287 , train accuracy = 0.9615 , valid loss = 0.1213 , valid accuracy = 0.9644
Epoch  9/100: 
train loss = 0.1195 , train accuracy = 0.9631 , valid loss = 0.1136 , valid accuracy = 0.9654
Epoch 10/100: 
trai

train loss = 0.0038 , train accuracy = 0.9988 , valid loss = 0.0489 , valid accuracy = 0.9880
Epoch 78/100: 
train loss = 0.0032 , train accuracy = 0.9993 , valid loss = 0.0452 , valid accuracy = 0.9884
Epoch 79/100: 
train loss = 0.0027 , train accuracy = 0.9993 , valid loss = 0.0486 , valid accuracy = 0.9882
Epoch 80/100: 
train loss = 0.0030 , train accuracy = 0.9992 , valid loss = 0.0485 , valid accuracy = 0.9880
Epoch 81/100: 
train loss = 0.0028 , train accuracy = 0.9994 , valid loss = 0.0466 , valid accuracy = 0.9880
Epoch 82/100: 
train loss = 0.0027 , train accuracy = 0.9993 , valid loss = 0.0493 , valid accuracy = 0.9882
Epoch 83/100: 
train loss = 0.0023 , train accuracy = 0.9997 , valid loss = 0.0458 , valid accuracy = 0.9882
Epoch 84/100: 
train loss = 0.0023 , train accuracy = 0.9994 , valid loss = 0.0537 , valid accuracy = 0.9868
Epoch 85/100: 
train loss = 0.0034 , train accuracy = 0.9988 , valid loss = 0.0541 , valid accuracy = 0.9870
Epoch 86/100: 
train loss = 0.0021

In [12]:
acc_val = sess.run(accuracy,feed_dict={X : test_image , y : test_label , keep_prob : 1})
print("acc = %.4f     "%(acc_val))

acc = 0.9894     
